# Parameter Shift and Finite Difference Gradients with Operator flow

In [1]:
import qiskit.aqua.operators as of
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit import Parameter
import numpy as np
from qiskit import BasicAer

#### Build Expectation Measurement

In [2]:
ansatz_op = of.PrimitiveOp(RealAmplitudes(3, reps=3))
op_params = ansatz_op.primitive.ordered_parameters
qs = ansatz_op.num_qubits
expect_op = ~of.StateFn(of.Z^qs) @ ansatz_op @ of.Zero

# Uncomment to try with basis changes
# expect_op = ~of.StateFn((of.X^qs) + (of.Z^qs) + (of.Y^qs) + (of.I^qs)) @ ansatz_op @ of.Zero
# expect_op = of.PauliExpectation().convert(expect_op)

# Point at which we're taking the gradient
grad_point = ((np.pi/4)*np.ones(len(op_params))).tolist()
param_dict = dict(zip(op_params, grad_point))

print(expect_op)

ComposedOp(
[OperatorMeasurement(ZZZ),
CircuitStateFn(
     ┌──────────┐          ┌──────────┐                      ┌──────────┐»
q_0: ┤ RY(θ[0]) ├──■────■──┤ RY(θ[3]) ├──────────────■────■──┤ RY(θ[6]) ├»
     ├──────────┤┌─┴─┐  │  └──────────┘┌──────────┐┌─┴─┐  │  └──────────┘»
q_1: ┤ RY(θ[1]) ├┤ X ├──┼───────■──────┤ RY(θ[4]) ├┤ X ├──┼───────■──────»
     ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    »
q_2: ┤ RY(θ[2]) ├─────┤ X ├───┤ X ├────┤ RY(θ[5]) ├─────┤ X ├───┤ X ├────»
     └──────────┘     └───┘   └───┘    └──────────┘     └───┘   └───┘    »
«                           ┌──────────┐             
«q_0: ──────────────■────■──┤ RY(θ[9]) ├─────────────
«     ┌──────────┐┌─┴─┐  │  └──────────┘┌───────────┐
«q_1: ┤ RY(θ[7]) ├┤ X ├──┼───────■──────┤ RY(θ[10]) ├
«     ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    ├───────────┤
«q_2: ┤ RY(θ[8]) ├─────┤ X ├───┤ X ├────┤ RY(θ[11]) ├
«     └──────────┘     └───┘   └───┘    └───────────┘
)])


#### Parameter Shift Gradients (Exact)

In [3]:
# Note: Doesn't handle any special cases
def param_shift_gradient(op_expr, params):
    grad_ops = []
    for param in params:
        grad_ops += [op_expr.bind_parameters({param: param + np.pi / 2}) -
                     op_expr.bind_parameters({param: param - np.pi / 2})]
    return .5 * of.ListOp(grad_ops)

In [4]:
ps_grad = param_shift_gradient(expect_op, op_params)
ps_grad_bound = ps_grad.bind_parameters(param_dict)
# print(ps_grad_bound)

ps_grad_vals = np.asarray(ps_grad_bound.eval())
print(ps_grad_vals)

[-0.55445752+0.j  0.08034587+0.j  0.2236517 +0.j -0.05981917+0.j
 -0.13840413+0.j  0.26784587+0.j -0.27542839+0.j  0.4736517 +0.j
 -0.12231917+0.j -0.24130704+0.j -0.30066626+0.j -0.38055217+0.j]


#### Finite (Forward and Central) Difference Gradients (Exact)

In [5]:
# Note: Central differences are better than forward differences!
def finite_difference_gradient(op_expr, params, step=.01, center=True):
    if center:
        grad_ops = [op_expr.bind_parameters({param: param + (step/2)}) - 
                    op_expr.bind_parameters({param: param - (step/2)})
                    for param in params]
        return of.ListOp(grad_ops) / step
    else:
        grad_ops = [op_expr.bind_parameters({param: param + step}) for param in params]
        return (of.ListOp(grad_ops) - op_expr) / step

In [6]:
fd_grad = finite_difference_gradient(expect_op, op_params, step=np.pi/8, center=True)
fd_grad_bound = fd_grad.bind_parameters(param_dict)
# print(fd_grad_bound)
fd_grad_vals = np.asarray(fd_grad_bound.eval())
print(fd_grad_vals)

[-0.5509017 +0.j  0.0798306 +0.j  0.22221738+0.j -0.05943554+0.j
 -0.13751652+0.j  0.26612813+0.j -0.27366203+0.j  0.4706141 +0.j
 -0.12153472+0.j -0.23975951+0.j -0.29873804+0.j -0.37811164+0.j]


In [7]:
# Approximation Error
grad_diffs = ps_grad_vals - fd_grad_vals
print(grad_diffs)
np.mean(np.abs(grad_diffs))

[-0.00355582+0.j  0.00051527+0.j  0.00143431+0.j -0.00038363+0.j
 -0.00088761+0.j  0.00171774+0.j -0.00176636+0.j  0.0030376 +0.j
 -0.00078445+0.j -0.00154754+0.j -0.00192822+0.j -0.00244054+0.j]


0.0016665898013804355

#### Circuit Sampling Instead of Exact Evaluation

In [8]:
# Trying with sampling
cs = of.CircuitSampler(BasicAer.get_backend('qasm_simulator'))
cs.quantum_instance.run_config.shots = 10000
sampled_ps_op = cs.convert(ps_grad_bound)
sampled_fd_op = cs.convert(fd_grad_bound)
sampled_ps_vals = np.asarray(sampled_ps_op.eval())
sampled_fd_vals = np.asarray(sampled_fd_op.eval())
print(sampled_ps_vals)
print(sampled_fd_vals)

[-0.02775+0.j -0.36135+0.j  0.0756 +0.j -0.03695+0.j -0.27215+0.j
 -0.1719 +0.j -0.06765+0.j -0.19155+0.j -0.0991 +0.j -0.00065+0.j
  0.00425+0.j  0.0014 +0.j]
[-7.19526306+0.j -7.45075456+0.j -7.17321557+0.j -7.2575148 +0.j
 -7.36126769+0.j -7.33273565+0.j -7.36645534+0.j -7.40665958+0.j
 -7.22638893+0.j -7.22509202+0.j -7.23287349+0.j -7.21212291+0.j]


In [9]:
# Approximation Error, Sampled PS
sampled_ps_grad_diff = ps_grad_vals - sampled_ps_vals
print(sampled_ps_grad_diff)
np.mean(np.abs(sampled_ps_grad_diff))

[-0.52670752+0.j  0.44169587+0.j  0.1480517 +0.j -0.02286917+0.j
  0.13374587+0.j  0.43974587+0.j -0.20777839+0.j  0.6652017 +0.j
 -0.02321917+0.j -0.24065704+0.j -0.30491626+0.j -0.38195217+0.j]


0.2947117279312563

In [10]:
# Approximation Error, Sampled FD (compare to PS unsampled, which is exact)
sampled_fd_grad_diff = ps_grad_vals - sampled_fd_vals
print(sampled_fd_grad_diff)
np.mean(np.abs(sampled_fd_grad_diff))

[6.64080554+0.j 7.53110043+0.j 7.39686727+0.j 7.19769563+0.j
 7.22286356+0.j 7.60058151+0.j 7.09102694+0.j 7.88031128+0.j
 7.10406976+0.j 6.98378498+0.j 6.93220723+0.j 6.83157074+0.j]


7.201073738163966